In [9]:
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install ibm_metrics_plugin --upgrade | tail -n 1
!pip install ibm_metrics_plugin --upgrade | tail -n 1
!pip install ibm_wos_utils --upgrade | tail -n 1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlmonitor 1.0.16 requires ibm-watson-machine-learning==1.0.321, but you have ibm-watson-machine-learning 1.0.347 which is incompatible.
mlmonitor 1.0.16 requires ipython==8.14.0, but you have ipython 8.18.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataclasses-json 0.5.14 requires marshmallow<4.0.0,>=3.18.0, but you have marshmallow 3.10.0 which is incompatible.
langchain 0.0.342 requires langchain-core<0.1,>=0.0.7, but you have langchain-core 0.1.22 which is incompatible.
trulens-eval 0.18.1 requires pydantic<2,>=1.10.7, but you have pydantic 2.5.2 which is incompatible.
trulens-eval 0.18.1 requires sqlalchemy>=2.0.19, but you have sqlalchemy 1.4.51 which is i

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
from dotenv import load_dotenv
load_dotenv("creds")
load_dotenv("assets/usecases/gcr/gcr_usecase.env")

True

In [3]:
from libs.WOS_Authenticator import WOSAuthenticator
from libs.WML_Authenticator import WMLAuthenticator
from libs.Reference_Data_Collector import ReferenceDataAnalyser
from libs.utils import save_as_json, read_from_json
from libs.Model_Subscriber import ModelSubscriber
from libs.Metrics_Subscriber import MetricsMonitor
from libs.Payload_Logging import PayloadLogging

# 1. OpenScale Authenticator

In [4]:
wos_authenticator = WOSAuthenticator()
wos_client = wos_authenticator.authenticate()
wml_authenticator = WMLAuthenticator()
wml_client = wml_authenticator.authenticate()


3.0.34
1.0.347


# 2. Collect Reference Data

In [5]:
analyser = ReferenceDataAnalyser()
df = analyser.collect_reference_data()
config_json = analyser.generate_reference_config()
df.head(5)

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


# 3. Subscribe Model

In [8]:

# Assuming wos_client is an instance of APIClient from ibm_watson_openscale
subscriber = ModelSubscriber(wos_client)
MLCredentials ={}
subscriber.setup_datamart()

subscriber.add_service_provider()
subscriber.subscribe_model(config_json)

subscriber.push_payload_record()
subscription_details = subscriber.verify_subscription()


None,None,False,active,2024-02-07 03:45:35.177000+00:00,00000000-0000-0000-0000-000000000000


None
Using existing datamart 00000000-0000-0000-0000-000000000000
Deleted existing service_provider with ID: 696c28b0-f9a7-4744-99d0-eeff96698bc8



 Waiting for end of adding service provider 26c0a8fe-400b-4ea4-8146-6c6d04d526b1 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------


Added new service provider with ID: 26c0a8fe-400b-4ea4-8146-6c6d04d526b1
Creating headless subscription as both deployment id and deployment space id are null



 Waiting for end of adding subscription 7664bcbb-82df-44d1-a30e-40b3e1182ac7 




preparing
active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------


Subscribed model with ID: 7664bcbb-82df-44d1-a30e-40b3e1182ac7
Payload data set id:  ddb0a7cd-2324-440c-92c2-ba97471b666e
Number of records in the payload logging table: 100


# 4. Subscribe Metrics

In [9]:
# wml_client.set.default_space(os.environ["SPACE_ID"])
monitor = MetricsMonitor(wos_client, wml_client, subscriber.data_mart_id, subscriber.subscription_id,analyser.config_json, analyser.class_label)

# Develop metrics
# ibm_cloud = False
if os.environ["IBM_CLOUD"] =="True":
    
    monitor.generate_access_token()
monitor.explanation_metric()
monitor.fairness_metric()
monitor.quality_metric()
monitor.drift_metric(analyser.feature_columns,analyser.cat_features)
monitor.custom_metric()


Single record scoring result: 
 fields: ['prediction', 'probability'] 
 values:  ['No Risk', [0.6833973497103111, 0.3166026502896889]]
{'probabilities': [[0.6833973497103111, 0.3166026502896889], [0.9547385853464627, 0.045261414653537224], [0.9895506471184039, 0.010449352881596149], [0.30388044172226325, 0.6961195582777367], [0.9796544066557112, 0.020345593344288756], [0.8132623980251094, 0.18673760197489064], [0.996122645597291, 0.0038773544027089274], [0.8837869673425738, 0.11621303265742625], [0.9929731220702196, 0.007026877929780353], [0.936099720074654, 0.063900279925346], [0.9681982589961272, 0.031801741003872765], [0.9820658472070485, 0.017934152792951553], [0.17558757920782697, 0.824412420792173], [0.7704372251667341, 0.22956277483326595], [0.8884971783508945, 0.11150282164910552], [0.947023832057638, 0.05297616794236201], [0.9878708487678869, 0.01212915123211311], [0.9797452709332584, 0.02025472906674168], [0.40379577157502333, 0.5962042284249767], [0.5689826069575248, 0.43101

00000000-0000-0000-0000-000000000000,active,7664bcbb-82df-44d1-a30e-40b3e1182ac7,subscription,fairness,2024-02-21 06:29:33.944000+00:00,07fe7ed8-9a22-4217-b97e-078690e44850
00000000-0000-0000-0000-000000000000,active,7664bcbb-82df-44d1-a30e-40b3e1182ac7,subscription,explainability,2024-02-21 06:28:18.135000+00:00,f736b975-c031-482a-bb96-f84f0349e5ae
00000000-0000-0000-0000-000000000000,active,7664bcbb-82df-44d1-a30e-40b3e1182ac7,subscription,model_health,2024-02-21 06:27:38.233000+00:00,0441fb37-c7cf-4f55-981f-e21f371a71ae
00000000-0000-0000-0000-000000000000,active,7664bcbb-82df-44d1-a30e-40b3e1182ac7,subscription,performance,2024-02-21 06:27:38.667000+00:00,13c6ded1-0ee1-48e1-881d-e97444525e2b
00000000-0000-0000-0000-000000000000,active,7087b751-c9e1-4164-8a38-0335ac8d88e5,subscription,mrm,2024-02-09 01:07:19.243000+00:00,6026ef5f-13fb-469e-bc6d-e2bad826328d
00000000-0000-0000-0000-000000000000,active,1bc1ed5c-fea0-4924-bfab-6762b7b779aa,subscription,fairness,2024-02-08 22:58:25.805000+00:00,d5ea43ed-2299-4c19-95ce-0071c72ad89e
00000000-0000-0000-0000-000000000000,active,1bc1ed5c-fea0-4924-bfab-6762b7b779aa,subscription,model_health,2024-02-08 06:55:53.140000+00:00,26d51146-2ba9-49d3-961e-537b7a6a26aa
00000000-0000-0000-0000-000000000000,active,1bc1ed5c-fea0-4924-bfab-6762b7b779aa,subscription,mrm,2024-02-08 06:55:53.227000+00:00,06497ff4-dfe3-4e3f-bc55-899e75228d2b
00000000-0000-0000-0000-000000000000,active,7087b751-c9e1-4164-8a38-0335ac8d88e5,subscription,fairness,2024-02-09 00:48:13.793000+00:00,cf29eda5-87a5-4b62-8d4a-468452aa2a76
00000000-0000-0000-0000-000000000000,active,7087b751-c9e1-4164-8a38-0335ac8d88e5,subscription,model_health,2024-02-09 00:44:24.448000+00:00,aee6c36f-e9d3-4bd8-8620-40a7fca948a4


Note: First 10 records were displayed.
run_status:  finished


2024-02-21 06:29:36.877327+00:00,fairness_value,293c7791-c726-4309-a26e-46906c73abff,39.583,98.0,None,"['feature:referrer_gender', 'fairness_metric_type:fairness', 'feature_value:male']",fairness,07fe7ed8-9a22-4217-b97e-078690e44850,f8aa2937-3844-488e-bb2f-c70a7aa40963,subscription,7664bcbb-82df-44d1-a30e-40b3e1182ac7





 Waiting for end of monitor instance creation afefa58f-c4fb-486f-9e39-9332267c4c4f 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------


/Users/skasmani/Downloads/personal/github/AI-Governance



 Waiting for end of storing records with request id: e9fabc83-c43a-4ea0-97d5-e3ea36e69d43 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------


10



 Waiting for end of monitoring run b4e88bd6-01ca-4188-8e55-051a5100df09 




error

-------------------------------
 Run failed with status: error 
-------------------------------


Reason: ['code: AIQFS0020E, message: 100 records are required to evaluate quality. 10 feedback records are available. Add 90 additional feedback records or lower the minimum number of records required by the quality monitor']


...

/Users/skasmani/opt/anaconda3/envs/ai_310/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/Users/skasmani/opt/anaconda3/envs/ai_310/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/Users/skasmani/opt/anaconda3/envs/ai_310/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


Learning two feature constraints...: 100%|██████████| 209/209 [00:01<00:00, 202.59constraints/s]
{
  "metadata": {
    "id": "0063948e-dc18-48bd-9737-b8dc2ff5d715",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:0063948e-dc18-48bd-9737-b8dc2ff5d715",
    "url": "/v2/monitor_instances/0063948e-dc18-48bd-9737-b8dc2ff5d715",
    "created_at": "2024-02-21T06:30:31.279000Z",
    "created_by": "saeed.kasmani@ibm.com"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "7664bcbb-82df-44d1-a30e-40b3e1182ac7"
    },
    "parameters": {
      "drift_threshold": 0.05,
      "min_samples": 100,
      "train_drift_model": false
    },
    "thresholds": [
      {
        "metric_id": "drift_magnitude",
        "type": "upper_limit",
        "value": 0.5
      },
      {
        "metric_id": "predict

2024-02-21 06:31:02.197493+00:00,data_drift_magnitude,a6ea806c-1b08-446f-8fb0-aaf5136b03aa,0.1,None,0.1,[],drift,0063948e-dc18-48bd-9737-b8dc2ff5d715,041bf2e9-0764-40cd-883f-6e53be22f270,subscription,7664bcbb-82df-44d1-a30e-40b3e1182ac7
2024-02-21 06:31:02.197493+00:00,drift_magnitude,a6ea806c-1b08-446f-8fb0-aaf5136b03aa,0.365,None,0.05,[],drift,0063948e-dc18-48bd-9737-b8dc2ff5d715,041bf2e9-0764-40cd-883f-6e53be22f270,subscription,7664bcbb-82df-44d1-a30e-40b3e1182ac7
2024-02-21 06:31:02.197493+00:00,predicted_accuracy,a6ea806c-1b08-446f-8fb0-aaf5136b03aa,0.6,None,None,[],drift,0063948e-dc18-48bd-9737-b8dc2ff5d715,041bf2e9-0764-40cd-883f-6e53be22f270,subscription,7664bcbb-82df-44d1-a30e-40b3e1182ac7


my_model_performance,my model performance,"['sensitivity', 'specificity']"
assurance,Assurance,"['Uncertainty', 'Confidence']"
fairness,Fairness,"['Fairness value', 'Average odds difference metric value', 'False discovery rate difference metric value', 'Error rate difference metric value', 'False negative rate difference metric value', 'False omission rate difference metric value', 'False positive rate difference metric value', 'True positive rate difference metric value', 'Average absolute odds difference metric value', 'Statistical parity difference metric value']"
model_health,Model Health,"['Total Scoring Requests', 'Total Records', 'Average Records', 'Median Records', 'Maximum Records', 'Minimum Records', 'Total Payload Size', 'Average Payload Size', 'Median Payload Size', 'Minimum Payload Size', 'Maximum Payload Size', 'Average API Throughput', 'Minimum API Throughput', 'Maximum API Throughput', 'Median API throughput', 'Average API Latency', 'Minimum API Latency', 'Maximum API Latency', 'Median API Latency', 'Average Record Throughput', 'Minimum Record Throughput', 'Maximum Record Throughput', 'Median Record Throughput', 'Average Record Latency', 'Minimum Record Latency', 'Maximum Record Latency', 'Median Record Latency', 'Users', 'Errors', 'Data Errors', 'System Errors']"
performance,Performance,['Number of records']
data_health,Data Health,"['Absence Count', 'Empty Strings', 'Data Type Mismatch', 'Class Confusion', 'Duplicate Rows', 'Unique Columns']"
explainability,Explainability,['Global explanation stability']
mrm,Model risk management monitoring,"['Tests run', 'Tests passed', 'Tests failed', 'Tests skipped', 'Fairness score', 'Quality score', 'Drift score']"
correlations,Correlations,"['Maximum positive correlation coefficient', 'Maximum negative correlation coefficient', 'Mean absolute correlation coefficient', 'Significant correlation coefficients count']"
drift_v2,Drift v2,"['Output drift', 'Model quality drift', 'Feature drift', 'Records processed', 'New records processed']"


Note: First 10 records were displayed.
my_model_performance
Monitor definition details: {
  "metadata": {
    "id": "my_model_performance",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_definition:my_model_performance",
    "url": "/v2/monitor_definitions/my_model_performance",
    "created_at": "2024-02-09T00:50:17.328000Z",
    "created_by": "saeed.kasmani@ibm.com"
  },
  "entity": {
    "name": "my model performance",
    "metrics": [
      {
        "name": "sensitivity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.8
          }
        ],
        "expected_direction": "increasing",
        "id": "sensitivity"
      },
      {
        "name": "specificity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.75
          }
        ],
        "expected_direction": "increasing",
        "id": "specificity"
      }
    ],
    "tags"

# 5. Payload Logging

In [10]:
inferencing = PayloadLogging(wos_client, wml_client, subscriber.subscription_id)

inferencing.log_payload()

Payload data set id:  ddb0a7cd-2324-440c-92c2-ba97471b666e
Performing explicit payload logging.
Number of records in the payload logging table: 1100


In [ ]:
wos_client.data_sets.get_records_count("377f396c-28c2-4caf-825b-d14f1d7c2bf1")